<a href="https://colab.research.google.com/github/doctorsmylie/mtg-draft-agent/blob/main/data_processing/gamedata_processing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This Notebook takes the 17Lands Gameplay data, processes it, and seves it to an HDF (.h5) file in the same folder

In [1]:
# Configure Drive or Jupyter notebook -- only runs when first loaded
if "CONFIG_DONE" not in globals():
    # Need to mount drive and clone repo to access data and functions
    try:
        from google.colab import drive  # type: ignore

        IN_COLAB = True

        # clone repo
        !git clone https://github.com/doctorsmylie/mtg-draft-agent
        %cd mtg-draft-agent

    except ModuleNotFoundError:
        IN_COLAB = False

    # Finish configuration -- also configures notebook outside of Colab
    %run "project_path_mod.ipynb"
else:
    print("Config done already")

fatal: destination path 'mtg-draft-agent' already exists and is not an empty directory.
/content/mtg-draft-agent
Starting config...
Running in Colab? Yes

Configuring Google Colab...
Mounting Drive...


/usr/local/lib/python3.11/dist-packages/nbformat/__init__.py:96: MissingIDFieldWarning: Cell is missing an id field, this will become a hard error in future nbformat versions. You may want to use `normalize()` on your notebooks before validations (available since nbformat 5.1.4). Previous versions of nbformat are fixing this issue transparently, and will stop doing so in the future.
  validate(nb)


Drive already mounted at /content/mtg-draft-agent/drive; to attempt to forcibly remount, call drive.mount("/content/mtg-draft-agent/drive", force_remount=True).
BASE_PATH =  /content/mtg-draft-agent
DATA_FOLDER = /content/mtg-draft-agent/drive/MyDrive/Erdos25/MTGdraft
BASE_PATH == os.getcwd(): True

Configuration done


In [2]:
import zipfile
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

# For managing paths
import pathlib

Loading data

In [3]:
# Expansion code
expansion = 'DSK'

In [5]:
def load_wr_gamedata(expansion):
  # load gamedata
  gamefilename='game_data_public.'+ expansion +'.PremierDraft.csv.gz'
  game_file = pathlib.Path(DATA_FOLDER, expansion, gamefilename)
  index_gamedata=pd.read_csv(game_file,compression='gzip',usecols=['draft_id','build_index','won'])
  nrows_index_gamedata=len(index_gamedata)

  # prompt: remove duplicates from deck_gamedata with the same 'draft_id' and 'build_index', creating a 'win_rate' column that aggregates the rows win rate
  index_gamedata['wins'] = index_gamedata['won'].astype(int)
  index_gamedata['row_number'] = np.arange(len(index_gamedata))+1
  winrate_agg = index_gamedata.groupby(['draft_id', 'build_index']).agg(
    wins=('wins', 'sum'),
    total_matches=('won', 'size'), # number of rows
    row_number=('row_number', 'max') # index of row
  )
  winrate_agg['win_rate'] = winrate_agg['wins'] / winrate_agg['total_matches']
  winrate_agg.sort_values(by=['row_number'], inplace=True)
  #filter for decks haven at least 2 matches
  winrate_agg=winrate_agg[winrate_agg['total_matches']>2]

  return winrate_agg

# #Test load_deck_gamedata
# wr_gamedata = load_wr_gamedata('DSK')
# print(wr_gamedata.shape)
# print(wr_gamedata.dtypes)

# #calculate the rows to include/exclude
# include_rows = [0] + wr_gamedata['row_number'].values.tolist()

# #print head
# wr_gamedata.head(10)



def load_deck_gamedata(expansion):
  #load WR data
  wr_gamedata = load_wr_gamedata(expansion)

  #calculate the rows to include
  include_rows = [0] + wr_gamedata['row_number'].values.tolist()
  #use set for faster lookup
  include_rows_set=set(include_rows)

  # load gamedata
  gamefilename='game_data_public.'+ expansion +'.PremierDraft.csv.gz'
  game_file = pathlib.Path(DATA_FOLDER, expansion, gamefilename)


  #load ONLY the rows from file corresponding to row_numbers in table
  load_decks= pd.read_csv(game_file,compression='gzip',skiprows=(lambda x: x not in include_rows_set))
  load_decks.drop(columns=['won'], inplace=True)
  load_decks.set_index(['draft_id','build_index'],inplace=True)

  #Renaming Basic Land Columns for them to stand out
  basic_lands_columns=list(load_decks.filter(regex='Plains$|Island$|Swamp$|Mountain$|Forest$').columns)
  #add '_Basic_Land' to these columns
  str_basic='_Basic_Land'
  basic_lands_dict={key : key+ str_basic for key in basic_lands_columns}
  load_decks.rename(columns=basic_lands_dict, inplace=True)

  complete_df=load_decks.merge(wr_gamedata, left_index=True, right_index=True)

  return complete_df


#Test load_deck_gamedata
deck_gamedata=load_deck_gamedata('DSK')
print(deck_gamedata.shape)
print(deck_gamedata.dtypes)
deck_gamedata.head(10)


(164413, 1451)
expansion                     object
event_type                    object
draft_time                    object
game_time                     object
match_number                   int64
                              ...   
user_game_win_rate_bucket    float64
wins                           int64
total_matches                  int64
row_number                     int64
win_rate                     float64
Length: 1451, dtype: object


,,expansion,event_type,draft_time,game_time,match_number,game_number,rank,opp_rank,main_colors,splash_colors,...,"drawn_Zimone, All-Questioning","tutored_Zimone, All-Questioning","deck_Zimone, All-Questioning","sideboard_Zimone, All-Questioning",user_n_games_bucket,user_game_win_rate_bucket,wins,total_matches,row_number,win_rate
draft_id,build_index,,,,,,,,,,,,,,,,,,,,,
53401b113a4f425fa26e60edd314dd27,0,DSK,PremierDraft,2024-09-24 18:48:56,2024-09-24 19:38:59,4,1,diamond,NaN,BR,NaN,...,0,0,0,0,500,0.58,1,4,4,0.250000
d26766b56fd14670ba60c2604bcd457b,0,DSK,PremierDraft,2024-09-24 20:10:55,2024-09-24 21:35:53,7,1,diamond,NaN,WU,NaN,...,0,0,0,0,500,0.58,4,7,11,0.571429
09e85424aa03464a96dd2b17c244f0b3,0,DSK,PremierDraft,2024-09-25 04:51:59,2024-09-25 05:49:55,4,1,diamond,NaN,RG,NaN,...,0,0,0,0,500,0.58,1,4,15,0.250000
e24bc7e44df94c138c9956cb6a7ac950,0,DSK,PremierDraft,2024-09-26 07:27:13,2024-09-26 08:54:17,9,1,diamond,NaN,WU,NaN,...,0,0,0,0,500,0.58,7,9,24,0.777778
e7e20fb8a2b9498e89bf0250648fb523,2,DSK,PremierDraft,2024-09-26 11:46:46,2024-09-26 12:59:48,6,1,diamond,NaN,UB,NaN,...,0,0,0,0,500,0.58,3,4,30,0.750000
ecd50bb4f01b4f5dbdac9f98cbf83b37,0,DSK,PremierDraft,2024-09-26 13:07:04,2024-09-26 14:15:00,8,1,diamond,NaN,WR,NaN,...,0,0,0,0,500,0.58,6,8,38,0.750000
ae2f97e5f8cc4a27b6eb5570d6791feb,0,DSK,PremierDraft,2024-09-27 02:06:40,2024-09-27 03:00:11,4,1,diamond,NaN,UBG,NaN,...,0,0,1,0,500,0.58,1,4,42,0.250000
dfe34397f8ea41418918465a4efb7329,0,DSK,PremierDraft,2024-09-27 03:06:43,2024-09-27 04:08:35,4,1,diamond,NaN,BG,U,...,0,0,0,0,500,0.58,1,4,46,0.250000
793e00680ec24ad086ea8cfa23eeeee2,0,DSK,PremierDraft,2024-09-30 05:34:12,2024-09-30 06:53:39,9,1,mythic,NaN,WR,B,...,0,0,0,0,500,0.58,6,9,55,0.666667


In [6]:
# Save deck_gamedata to h5 file
def save_deck_gamedata_h5(expansion, deck_gamedata):
  output_filename= expansion + '_processed_gamedata.h5'
  out_filepath = pathlib.Path(DATA_FOLDER, expansion, output_filename)
  deck_gamedata.to_hdf(out_filepath, key='deck_gamedata', mode='w', complib='zlib', complevel=9)
  print(f"Saved {expansion} deck gamedata to {output_filename}")

# TEST save_wr_gamedata
save_deck_gamedata_h5('DSK', deck_gamedata)

/tmp/ipython-input-6-3626640430.py:5: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block3_values] [items->Index(['expansion', 'event_type', 'draft_time', 'game_time', 'rank',
       'main_colors', 'splash_colors', 'opp_colors'],
      dtype='object')]

  deck_gamedata.to_hdf(out_filepath, key='deck_gamedata', mode='w', complib='zlib', complevel=9)


Saved DSK deck gamedata to DSK_processed_gamedata.h5


In [4]:
# prompt: make a function that loads wr_gamedata from expansion+'_processed_gamedata.h5'

def load_deck_gamedata_h5(expansion):
  # Construct the filename
  filename = expansion + '_processed_gamedata.h5'
  # Construct the full path
  file_path = pathlib.Path(DATA_FOLDER, expansion, filename)
  # Load the data from the HDF5 file
  deck_gamedata = pd.read_hdf(file_path, key='deck_gamedata')
  return deck_gamedata

# Test the function
deck_gamedata = load_deck_gamedata_h5('DSK')
print(deck_gamedata.shape)
print(deck_gamedata.dtypes)

# Print head
deck_gamedata.head(10)

(164413, 1451)
expansion                     object
event_type                    object
draft_time                    object
game_time                     object
match_number                   int64
                              ...   
user_game_win_rate_bucket    float64
wins                           int64
total_matches                  int64
row_number                     int64
win_rate                     float64
Length: 1451, dtype: object


,,expansion,event_type,draft_time,game_time,match_number,game_number,rank,opp_rank,main_colors,splash_colors,...,"drawn_Zimone, All-Questioning","tutored_Zimone, All-Questioning","deck_Zimone, All-Questioning","sideboard_Zimone, All-Questioning",user_n_games_bucket,user_game_win_rate_bucket,wins,total_matches,row_number,win_rate
draft_id,build_index,,,,,,,,,,,,,,,,,,,,,
53401b113a4f425fa26e60edd314dd27,0,DSK,PremierDraft,2024-09-24 18:48:56,2024-09-24 19:38:59,4,1,diamond,NaN,BR,NaN,...,0,0,0,0,500,0.58,1,4,4,0.250000
d26766b56fd14670ba60c2604bcd457b,0,DSK,PremierDraft,2024-09-24 20:10:55,2024-09-24 21:35:53,7,1,diamond,NaN,WU,NaN,...,0,0,0,0,500,0.58,4,7,11,0.571429
09e85424aa03464a96dd2b17c244f0b3,0,DSK,PremierDraft,2024-09-25 04:51:59,2024-09-25 05:49:55,4,1,diamond,NaN,RG,NaN,...,0,0,0,0,500,0.58,1,4,15,0.250000
e24bc7e44df94c138c9956cb6a7ac950,0,DSK,PremierDraft,2024-09-26 07:27:13,2024-09-26 08:54:17,9,1,diamond,NaN,WU,NaN,...,0,0,0,0,500,0.58,7,9,24,0.777778
e7e20fb8a2b9498e89bf0250648fb523,2,DSK,PremierDraft,2024-09-26 11:46:46,2024-09-26 12:59:48,6,1,diamond,NaN,UB,NaN,...,0,0,0,0,500,0.58,3,4,30,0.750000
ecd50bb4f01b4f5dbdac9f98cbf83b37,0,DSK,PremierDraft,2024-09-26 13:07:04,2024-09-26 14:15:00,8,1,diamond,NaN,WR,NaN,...,0,0,0,0,500,0.58,6,8,38,0.750000
ae2f97e5f8cc4a27b6eb5570d6791feb,0,DSK,PremierDraft,2024-09-27 02:06:40,2024-09-27 03:00:11,4,1,diamond,NaN,UBG,NaN,...,0,0,1,0,500,0.58,1,4,42,0.250000
dfe34397f8ea41418918465a4efb7329,0,DSK,PremierDraft,2024-09-27 03:06:43,2024-09-27 04:08:35,4,1,diamond,NaN,BG,U,...,0,0,0,0,500,0.58,1,4,46,0.250000
793e00680ec24ad086ea8cfa23eeeee2,0,DSK,PremierDraft,2024-09-30 05:34:12,2024-09-30 06:53:39,9,1,mythic,NaN,WR,B,...,0,0,0,0,500,0.58,6,9,55,0.666667
